In [1]:
import pandas as pd
from collections import Counter
import os


# Select manually labelled samples for validation and test set

# get set of manually labelled texts

In [2]:
df_manual =pd.read_csv("Sensitive_topics_crowdsource_only.csv")

In [3]:
df_manual.head(2)

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice
0,"Нееет, родаки обязаны родив личинку дать ей об...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.19,0.0,1.0,1.0,1.0
1,баба в мусульманских странах она не имеет прав...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,1.0,1.0


In [4]:
manual_set = set(df_manual['text'])

# read final version of dataset

In [5]:
df_all = pd.read_csv("Sensitive_topics.csv")

In [6]:
df_all.head()

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice
0,РРРРРРЯЯЯЯЯЯЯ РУССКИЕ ВАНЬКИ НИКОМУ НЕ НУЖНЫ!!...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,"По моему, быдло тот кто осуждает чужие предпоч...",0.0,0.0,0.0,0.0,1.0,0.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Бабы это вообще безхуйные сиськастые ссущие си...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,изнасиловалиа говорила что девственницаЯ так д...,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,"и что ? не сосут только уебищные тни, видосов ...",0.0,0.0,0.0,0.0,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# look for intersections

In [7]:
def is_manual(text):
    if text in manual_set:
        return 'crowd_source'
    else:
        return 'semi_automative'

In [8]:
len(df_all)

25679

In [9]:
df_all['crowling'] = df_all['text'].apply(is_manual)

In [10]:
Counter(df_all['crowling'])

Counter({'semi_automative': 17504, 'crowd_source': 8175})

# Stratify

In [11]:
topics = list(df_all.columns)[1:-1]
topics

['offline_crime',
 'online_crime',
 'drugs',
 'gambling',
 'pornography',
 'prostitution',
 'slavery',
 'suicide',
 'terrorism',
 'weapons',
 'body_shaming',
 'health_shaming',
 'politics',
 'racism',
 'religion',
 'sexual_minorities',
 'sexism',
 'social_injustice']

In [12]:
df_manual_detected = df_all[df_all['crowling'] == "crowd_source"]
df_semi_auto = df_all[df_all['crowling'] == "semi_automative"]

In [13]:
len(df_manual_detected)

8175

In [14]:
is_multilabel = []
multilabel_samples_count = 0
for i,el in df_manual_detected.iterrows():
    current_non_null_topics_count = 0
    for t in topics:
        if el[t] >= 0.5:
            current_non_null_topics_count += 1
    if current_non_null_topics_count > 1:
        multilabel_samples_count += 1
        is_multilabel.append(current_non_null_topics_count)
    elif current_non_null_topics_count == 1:
        is_multilabel.append(1)
    else:
        is_multilabel.append(0)

In [15]:
df_manual_detected['multiclass'] = is_multilabel

<ipython-input-15-8386df71f46c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_manual_detected['multiclass'] = is_multilabel


In [16]:
df_manual_detected.head()

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,...,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,crowling,multiclass
2,Бабы это вообще безхуйные сиськастые ссущие си...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,crowd_source,1
5,Взгляните на пизду. Как психически здоровый че...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,crowd_source,3
8,"Нахуй мне баба? Жить мне одному удобнее, ебать...",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,crowd_source,2
12,"Т е ты совсем не общаешься с парнем, ми или му...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,crowd_source,2
13,Ну на публике она позиционирует себя как девоч...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,crowd_source,2


# We will use only manually labelled samples for validation and test

In [17]:
df_multi_none = df_manual_detected[df_manual_detected['multiclass'] == 0]
df_multi_mono = df_manual_detected[df_manual_detected['multiclass'] == 1]
df_multi_many = df_manual_detected[df_manual_detected['multiclass'] > 1]

In [18]:
split_train = 0.7
split_val = 0.85

RANDOM_STATE = 0

# Split samples with no topics


In [19]:
df_multi_none = df_multi_none.sample(frac=1, random_state = RANDOM_STATE).reset_index(drop=True)

In [20]:
split_index_train = int(len(df_multi_none)*split_train)
split_index_val = int(len(df_multi_none)*split_val)

df_multi_none_train = df_multi_none[:split_index_train]
df_multi_none_val = df_multi_none[split_index_train:split_index_val]
df_multi_none_test = df_multi_none[split_index_val:]

# Split one label samples


In [21]:
df_multi_mono_train_list = []
df_multi_mono_val_list = []
df_multi_mono_test_list = []

for t in topics:
    df_mono_current_topic = df_multi_mono[df_multi_mono[t] >= 0.5]
    total_lens += len(df_mono_current_topic)
    df_mono_current_topic = df_mono_current_topic.sample(frac=1, random_state = RANDOM_STATE).reset_index(drop=True)
    
    split_index_train = int(len(df_mono_current_topic)*split_train)
    split_index_val = int(len(df_mono_current_topic)*split_val)
    
    df_mono_current_topic_train = df_mono_current_topic[:split_index_train]
    df_multi_mono_train_list.append(df_mono_current_topic_train)
    
    df_mono_current_topic_val = df_mono_current_topic[split_index_train:split_index_val]
    df_multi_mono_val_list.append(df_mono_current_topic_val)
    
    df_mono_current_topic_test = df_mono_current_topic[split_index_val:]
    df_multi_mono_test_list.append(df_mono_current_topic_test)
    
#     break
    
df_multi_mono_train = pd.concat(df_multi_mono_train_list)
df_multi_mono_val = pd.concat(df_multi_mono_val_list)
df_multi_mono_test = pd.concat(df_multi_mono_test_list)



In [36]:
len(df_multi_mono_train) + len(df_multi_mono_val) + len(df_multi_mono_test), len(df_multi_mono)

(3957, 3957)

# Adjust multilable samples

In [38]:
one_sample_labels = []
for i,el in df_multi_many.iterrows():
    current_el_classes = []
    for t in topics:
        if el[t] > 0.5:
            current_el_classes.append(t)
    current_el_classes = sorted(current_el_classes)
    current_el_classes = '='.join(current_el_classes)
    one_sample_labels.append(current_el_classes)


In [39]:
c = Counter(one_sample_labels)
multisamples_counter_dict = dict(c)
c.most_common(10)

[('pornography=sexual_minorities', 234),
 ('politics=racism=social_injustice', 215),
 ('politics=racism', 182),
 ('body_shaming=pornography', 113),
 ('pornography=sexism', 108),
 ('prostitution=sexism', 87),
 ('politics=social_injustice', 60),
 ('politics=weapons', 55),
 ('politics=terrorism', 51),
 ('offline_crime=politics', 48)]

In [40]:
df_multi_many['multiclass'] = one_sample_labels

<ipython-input-40-71733ff6f017>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_multi_many['multiclass'] = one_sample_labels


# Apply same stratification logic as with one label samples

In [45]:
df_multi_many_train_list = []
df_multi_many_val_list = []
df_multi_many_test_list = []

total_count = 0

for multi_topics,count in multisamples_counter_dict.items():
    df_multi_current_topic = df_multi_many[df_multi_many['multiclass'] ==multi_topics]
    
    try:
        len(df_multi_current_topic) == count
    except Exception as e:
        print(multi_topics,count, len(df_multi_current_topic))
        
    total_count += count
    if count < 2:
        df_multi_many_train_list.append(df_multi_current_topic)
    else:
        df_multi_current_topic = df_multi_current_topic.sample(frac=1, random_state = RANDOM_STATE).reset_index(drop=True)
    
        split_index_train = int(len(df_multi_current_topic)*split_train)
        split_index_val = int(len(df_multi_current_topic)*split_val)

        df_multi_current_topic_train = df_multi_current_topic[:split_index_train]
        df_multi_many_train_list.append(df_multi_current_topic_train)

        df_multi_current_topic_val = df_multi_current_topic[split_index_train:split_index_val]
        df_multi_many_val_list.append(df_multi_current_topic_val)
        
        df_multi_current_topic_test = df_multi_current_topic[split_index_val:]
        df_multi_many_test_list.append(df_multi_current_topic_test)
    
df_multi_many_train = pd.concat(df_multi_many_train_list)
df_multi_many_val = pd.concat(df_multi_many_val_list)    
df_multi_many_test = pd.concat(df_multi_many_test_list) 


In [46]:
len(df_multi_many_train) + len(df_multi_many_val) + len(df_multi_many_test), len(df_multi_many)

(2838, 2838)

In [47]:
len(df_multi_many_train), len(df_multi_many_val), round(len(df_multi_many_val)/ (len(df_multi_many_train) + len(df_multi_many_val)),2)

(1952, 375, 0.16)

In [48]:
necessary_columns = ['text'] + topics

In [49]:
df_multi_mono_train = df_multi_mono_train[necessary_columns]
df_multi_mono_val = df_multi_mono_val[necessary_columns]
df_multi_mono_test = df_multi_mono_test[necessary_columns]

df_multi_none_train = df_multi_none_train[necessary_columns]
df_multi_none_val = df_multi_none_val[necessary_columns]
df_multi_none_test = df_multi_none_test[necessary_columns]

df_multi_many_train = df_multi_many_train[necessary_columns]
df_multi_many_val = df_multi_many_val[necessary_columns]
df_multi_many_test = df_multi_many_test[necessary_columns]

In [50]:
df_train = pd.concat([df_multi_mono_train,df_multi_many_train, df_multi_none_train ])
df_train.drop_duplicates(inplace=True, subset = ['text'], keep = 'first')

df_val = pd.concat([df_multi_mono_val,df_multi_many_val,df_multi_none_val ])
df_val.drop_duplicates(inplace=True, subset = ['text'], keep = 'first')

df_test = pd.concat([df_multi_mono_test,df_multi_none_test,df_multi_many_test])
df_test.drop_duplicates(inplace=True, subset = ['text'], keep = 'first')

In [51]:
len(df_train) + len(df_val) + len(df_test), len(df_manual_detected)

(8175, 8175)

In [52]:
len(df_manual_detected), len(df_train), len(df_val), round(len(df_val)/ (len(df_train) + len(df_val)),2)

(8175, 5679, 1174, 0.17)

# Sanity check

In [54]:
df_assert = pd.concat([df_train, df_val, df_test]) 
len(df_assert['text']) , len(set(df_assert['text']))

(8175, 8175)

# Concatenate with semi-automative dataset

In [55]:
df_assert.columns

Index(['text', 'offline_crime', 'online_crime', 'drugs', 'gambling',
       'pornography', 'prostitution', 'slavery', 'suicide', 'terrorism',
       'weapons', 'body_shaming', 'health_shaming', 'politics', 'racism',
       'religion', 'sexual_minorities', 'sexism', 'social_injustice'],
      dtype='object')

In [56]:
df_semi_auto.columns

Index(['text', 'offline_crime', 'online_crime', 'drugs', 'gambling',
       'pornography', 'prostitution', 'slavery', 'suicide', 'terrorism',
       'weapons', 'body_shaming', 'health_shaming', 'politics', 'racism',
       'religion', 'sexual_minorities', 'sexism', 'social_injustice',
       'crowling'],
      dtype='object')

In [57]:
df_semi_auto = df_semi_auto[necessary_columns]

In [58]:
df_train_all = pd.concat([df_train,df_semi_auto])

In [60]:
len(df_train_all) + len(df_val) + len(df_test), len(df_all), round(len(df_val)/len(df_all),2),round(len(df_test)/len(df_all),2)

(25679, 25679, 0.05, 0.05)

# Round values by threshold


In [62]:
ROUND_THRESHOLD = 0.75

def round_val(val):
    if val < ROUND_THRESHOLD:
        return 0
    else:
        return 1

In [63]:
for df_curr in [df_train_all,df_val,df_test]:
    for clmn in df_curr.columns:
        if clmn != 'text':
            df_curr[clmn] = df_curr[clmn].apply(round_val)

In [65]:
df_train_all.to_csv('train.csv', index = None)
df_val.to_csv('val.csv', index = None)
df_test.to_csv('test.csv', index = None)